**July 2, 2025 • 04:58 PM**

# **Urgent Return vs. Planned Return Proportion**

__Continue from yesterday presentation (July 1, 2025 • 03:00 PM - 03:10 PM)__

> Remove the Return category of "COVID-19" and "โครงการป้อนงานคุณแม่บ้าน" to see the only 2 proportions of "Urgent" and "Planned" of service provider's return on booking to be able to see its trend more clearly

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
#import pymysql as pydb
#from dotenv import load_dotenv
import os
#from sqlalchemy import create_engine, text, Column, Integer, String, DateTime
#from sqlalchemy.orm import sessionmaker, declarative_base
from IPython.display import display, clear_output

import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from scipy.stats import norm

import warnings
import matplotlib.ticker as ticker
warnings.filterwarnings("ignore")

#### **Load Raw Data**

In [3]:
cancel_problem_pivot = pd.read_csv("cancel_problem_raw.csv")

# Drop 1st column which is an index column
cancel_problem_pivot = cancel_problem_pivot.drop(cancel_problem_pivot.columns[0], axis=1)
cancel_problem_pivot

,report_month,day_of_week,คืนงาน (กะทันหัน),คืนงาน (ล่วงหน้า)
0,2024-01,Fri,36,17
1,2024-01,Mon,22,23
2,2024-01,Sat,34,16
3,2024-01,Sun,26,6
4,2024-01,Thu,25,23
...,...,...,...,...
114,2025-05,Sat,38,27
115,2025-05,Sun,32,16
116,2025-05,Thu,36,19
117,2025-05,Tue,31,17


#### **Plot Visualizations**

In [4]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

df = cancel_problem_pivot.copy()

# Define day order from Sunday to Saturday
#day_order = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
day_order = ['Sat', 'Fri', 'Thu', 'Wed', 'Tue', 'Mon', 'Sun']
df['day_of_week'] = pd.Categorical(df['day_of_week'], categories=day_order, ordered=True)

# Sort DataFrame
df = df.sort_values(["report_month", "day_of_week"])

# Calculate total returns
df["total_returns"] = df["คืนงาน (กะทันหัน)"] + df["คืนงาน (ล่วงหน้า)"]

# Create pivot tables for heatmaps and reindex to ensure proper order
urgent_pivot = df.pivot(index='day_of_week', columns='report_month', values='คืนงาน (กะทันหัน)')
urgent_pivot = urgent_pivot.reindex(day_order)

planned_pivot = df.pivot(index='day_of_week', columns='report_month', values='คืนงาน (ล่วงหน้า)')
planned_pivot = planned_pivot.reindex(day_order)

total_pivot = df.pivot(index='day_of_week', columns='report_month', values='total_returns')
total_pivot = total_pivot.reindex(day_order)

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Urgent Returns (กะทันหัน)', 'Planned Returns (ล่วงหน้า)', 
                   'Total Returns', 'Monthly Trends by Day of Week'),
    specs=[[{"type": "heatmap"}, {"type": "heatmap"}],
           [{"type": "heatmap"}, {"type": "scatter"}]]
)

# Heatmap 1: Urgent Returns
fig.add_trace(
    go.Heatmap(
        z=urgent_pivot.values,
        x=urgent_pivot.columns,
        y=urgent_pivot.index,
        colorscale='Reds',
        name='Urgent Returns',
        hovertemplate='Month: %{x}<br>Day: %{y}<br>Count: %{z}<extra></extra>'
    ),
    row=1, col=1
)

# Heatmap 2: Planned Returns
fig.add_trace(
    go.Heatmap(
        z=planned_pivot.values,
        x=planned_pivot.columns,
        y=planned_pivot.index,
        colorscale='Blues',
        name='Planned Returns',
        hovertemplate='Month: %{x}<br>Day: %{y}<br>Count: %{z}<extra></extra>'
    ),
    row=1, col=2
)

# Heatmap 3: Total Returns
fig.add_trace(
    go.Heatmap(
        z=total_pivot.values,
        x=total_pivot.columns,
        y=total_pivot.index,
        colorscale='Viridis',
        name='Total Returns',
        hovertemplate='Month: %{x}<br>Day: %{y}<br>Count: %{z}<extra></extra>'
    ),
    row=2, col=1
)

# Line plot: Monthly trends by day of week
for day in day_order:
    day_data = df[df['day_of_week'] == day].sort_values('report_month')
    fig.add_trace(
        go.Scatter(
            x=day_data['report_month'],
            y=day_data['total_returns'],
            mode='lines+markers',
            name=day,
            line=dict(width=2),
            marker=dict(size=6),
            hovertemplate=f'{day}: %{{y}} returns<br>Month: %{{x}}<extra></extra>'
        ),
        row=2, col=2
    )

# Update layout
fig.update_layout(
    title_text="Work Returns Analysis - Monthly and Day of Week Trends",
    title_x=0.5,
    height=800,
    showlegend=True,
    font=dict(size=10)
)

# Update x-axis for all subplots to rotate labels
fig.update_xaxes(tickangle=45)

# Show the plot
fig.show()

# Additional analysis: Summary statistics
print("=== SUMMARY STATISTICS ===")
print("\nMonthly totals:")
monthly_summary = df.groupby('report_month').agg({
    'คืนงาน (กะทันหัน)': 'sum',
    'คืนงาน (ล่วงหน้า)': 'sum',
    'total_returns': 'sum'
}).round(2)
print(monthly_summary)

print("\nDay of week averages:")
daily_summary = df.groupby('day_of_week').agg({
    'คืนงาน (กะทันหัน)': 'mean',
    'คืนงาน (ล่วงหน้า)': 'mean',
    'total_returns': 'mean'
}).round(2)
print(daily_summary)

# Create a separate detailed line chart for better trend analysis
fig2 = go.Figure()

# Add lines for each day of week
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
for i, day in enumerate(day_order):
    day_data = df[df['day_of_week'] == day].sort_values('report_month')
    
    # Urgent returns
    fig2.add_trace(
        go.Scatter(
            x=day_data['report_month'],
            y=day_data['คืนงาน (กะทันหัน)'],
            mode='lines+markers',
            name=f'{day} - Urgent',
            line=dict(color=colors[i], dash='solid'),
            marker=dict(size=6, symbol='circle')
        )
    )
    
    # Planned returns
    fig2.add_trace(
        go.Scatter(
            x=day_data['report_month'],
            y=day_data['คืนงาน (ล่วงหน้า)'],
            mode='lines+markers',
            name=f'{day} - Planned',
            line=dict(color=colors[i], dash='dash'),
            marker=dict(size=6, symbol='square')
        )
    )

fig2.update_layout(
    title="Detailed Trends: Urgent vs Planned Returns by Day of Week",
    xaxis_title="Month",
    yaxis_title="Number of Returns",
    height=600,
    hovermode='x unified',
    xaxis=dict(tickangle=45)
)

fig2.show()

# Create a box plot to show distribution patterns
fig3 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Urgent Returns Distribution', 'Planned Returns Distribution')
)

# Box plot for urgent returns
for day in day_order:
    day_data = df[df['day_of_week'] == day]['คืนงาน (กะทันหัน)']
    fig3.add_trace(
        go.Box(
            y=day_data,
            name=day,
            boxpoints='all',
            jitter=0.3,
            pointpos=-1.8
        ),
        row=1, col=1
    )

# Box plot for planned returns
for day in day_order:
    day_data = df[df['day_of_week'] == day]['คืนงาน (ล่วงหน้า)']
    fig3.add_trace(
        go.Box(
            y=day_data,
            name=day,
            boxpoints='all',
            jitter=0.3,
            pointpos=-1.8,
            showlegend=False
        ),
        row=1, col=2
    )

fig3.update_layout(
    title="Distribution of Returns by Day of Week",
    height=500
)

fig3.show()

=== SUMMARY STATISTICS ===

Monthly totals:
              คืนงาน (กะทันหัน)  คืนงาน (ล่วงหน้า)  total_returns
report_month                                                     
2024-01                     194                122            316
2024-02                     201                101            302
2024-03                     235                117            352
2024-04                     187                111            298
2024-05                     213                115            328
2024-06                     218                 94            312
2024-07                     222                122            344
2024-08                     215                111            326
2024-09                     185                106            291
2024-10                     253                114            367
2024-11                     206                163            369
2024-12                     200                171            371
2025-01                     210 

#### **May 2025 Investigation Report**

In [5]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

df['total_returns'] = df['คืนงาน (กะทันหัน)'] + df['คืนงาน (ล่วงหน้า)']
day_order = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
df['day_of_week'] = pd.Categorical(df['day_of_week'], categories=day_order, ordered=True)

# Extract year and month for better analysis
df['year'] = df['report_month'].str[:4]
df['month'] = df['report_month'].str[5:7]
df['month_name'] = pd.to_datetime(df['report_month'] + '-01').dt.strftime('%b %Y')

print("🚨 MAY 2025 INVESTIGATION REPORT 🚨")
print("="*50)

# 1. MAY 2025 SPECIFIC ANALYSIS
may_2025_data = df[df['report_month'] == '2025-05'].copy()
print("\n📊 MAY 2025 DETAILED BREAKDOWN:")
print(may_2025_data[['day_of_week', 'คืนงาน (กะทันหัน)', 'คืนงาน (ล่วงหน้า)', 'total_returns']].to_string(index=False))

# Focus on Friday and Weekend (Fri, Sat, Sun)
weekend_days = ['Fri', 'Sat', 'Sun']
may_weekend = may_2025_data[may_2025_data['day_of_week'].isin(weekend_days)]
print(f"\n🔴 MAY 2025 WEEKEND + FRIDAY TOTAL: {may_weekend['total_returns'].sum()} returns")
print(f"   - Friday: {may_2025_data[may_2025_data['day_of_week']=='Fri']['total_returns'].values[0]} returns")
print(f"   - Saturday: {may_2025_data[may_2025_data['day_of_week']=='Sat']['total_returns'].values[0]} returns")
print(f"   - Sunday: {may_2025_data[may_2025_data['day_of_week']=='Sun']['total_returns'].values[0]} returns")

# 2. HISTORICAL COMPARISON
print("\n📈 HISTORICAL COMPARISON:")

# Get all May data for comparison
all_may_data = df[df['month'] == '05'].copy()
print("\nAll MAY months comparison:")
may_comparison = all_may_data.groupby(['report_month', 'day_of_week'])['total_returns'].sum().reset_index()
may_pivot = may_comparison.pivot(index='day_of_week', columns='report_month', values='total_returns')
print(may_pivot.loc[weekend_days])

# Calculate percentage increase
may_2024_weekend = df[(df['report_month'] == '2024-05') & (df['day_of_week'].isin(weekend_days))]['total_returns'].sum()
may_2025_weekend = df[(df['report_month'] == '2025-05') & (df['day_of_week'].isin(weekend_days))]['total_returns'].sum()
weekend_increase = ((may_2025_weekend - may_2024_weekend) / may_2024_weekend) * 100

print(f"\n🚨 KEY FINDINGS:")
print(f"   - May 2024 Weekend+Friday: {may_2024_weekend} returns")
print(f"   - May 2025 Weekend+Friday: {may_2025_weekend} returns")
print(f"   - INCREASE: {weekend_increase:.1f}%")

# 3. VISUALIZATIONS FOR C-LEVEL PRESENTATION

# Create focused analysis charts
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('May 2025 vs Historical Fridays', 'May 2025 vs Historical Weekends',
                   'May 2025 Daily Breakdown', 'Urgent vs Planned Returns Pattern'),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# Chart 1: Friday comparison across all years
friday_data = df[df['day_of_week'] == 'Fri'].copy()
colors_fri = ['red' if x == '2025-05' else 'lightblue' for x in friday_data['report_month']]
fig.add_trace(
    go.Bar(
        x=friday_data['report_month'],
        y=friday_data['total_returns'],
        name='Friday Returns',
        marker_color=colors_fri,
        text=friday_data['total_returns'],
        textposition='outside'
    ),
    row=1, col=1
)

# Chart 2: Weekend comparison
weekend_monthly = df[df['day_of_week'].isin(['Sat', 'Sun'])].groupby('report_month')['total_returns'].sum().reset_index()
colors_weekend = ['red' if x == '2025-05' else 'lightgreen' for x in weekend_monthly['report_month']]
fig.add_trace(
    go.Bar(
        x=weekend_monthly['report_month'],
        y=weekend_monthly['total_returns'],
        name='Weekend Returns',
        marker_color=colors_weekend,
        text=weekend_monthly['total_returns'],
        textposition='outside'
    ),
    row=1, col=2
)

# Chart 3: May 2025 daily breakdown
fig.add_trace(
    go.Bar(
        x=may_2025_data['day_of_week'],
        y=may_2025_data['total_returns'],
        name='May 2025 Daily',
        marker_color=['red' if x in weekend_days else 'blue' for x in may_2025_data['day_of_week']],
        text=may_2025_data['total_returns'],
        textposition='outside'
    ),
    row=2, col=1
)

# Chart 4: Urgent vs Planned pattern for May 2025
fig.add_trace(
    go.Bar(
        x=may_2025_data['day_of_week'],
        y=may_2025_data['คืนงาน (กะทันหัน)'],
        name='Urgent',
        marker_color='red',
        text=may_2025_data['คืนงาน (กะทันหัน)'],
        textposition='outside'
    ),
    row=2, col=2
)

fig.add_trace(
    go.Bar(
        x=may_2025_data['day_of_week'],
        y=may_2025_data['คืนงาน (ล่วงหน้า)'],
        name='Planned',
        marker_color='orange',
        text=may_2025_data['คืนงาน (ล่วงหน้า)'],
        textposition='outside'
    ),
    row=2, col=2
)

fig.update_layout(
    title_text="🚨 MAY 2025 WORK RETURNS INVESTIGATION - C-LEVEL BRIEFING",
    height=800,
    showlegend=True
)

fig.update_xaxes(tickangle=45)
fig.show()

# 4. STATISTICAL ANALYSIS
print("\n📊 STATISTICAL ANALYSIS:")

# Calculate z-scores to identify outliers
friday_history = df[df['day_of_week'] == 'Fri']['total_returns']
friday_mean = friday_history.mean()
friday_std = friday_history.std()
may_2025_fri = may_2025_data[may_2025_data['day_of_week'] == 'Fri']['total_returns'].values[0]
friday_z_score = (may_2025_fri - friday_mean) / friday_std

print(f"Friday Analysis:")
print(f"  - Historical Friday average: {friday_mean:.1f}")
print(f"  - May 2025 Friday: {may_2025_fri}")
print(f"  - Z-score: {friday_z_score:.2f} (>2 = outlier)")
print(f"  - Status: {'🚨 OUTLIER' if abs(friday_z_score) > 2 else '✅ Normal'}")

# Weekend analysis
weekend_history = df[df['day_of_week'].isin(['Sat', 'Sun'])].groupby('report_month')['total_returns'].sum()
weekend_mean = weekend_history.mean()
weekend_std = weekend_history.std()
may_2025_weekend_total = may_2025_data[may_2025_data['day_of_week'].isin(['Sat', 'Sun'])]['total_returns'].sum()
weekend_z_score = (may_2025_weekend_total - weekend_mean) / weekend_std

print(f"\nWeekend Analysis:")
print(f"  - Historical weekend average: {weekend_mean:.1f}")
print(f"  - May 2025 weekend: {may_2025_weekend_total}")
print(f"  - Z-score: {weekend_z_score:.2f}")
print(f"  - Status: {'🚨 OUTLIER' if abs(weekend_z_score) > 2 else '✅ Normal'}")

# 5. INVESTIGATION RECOMMENDATIONS
print("\n" + "="*50)
print("🔍 RECOMMENDED INVESTIGATION ACTIONS:")
print("="*50)

investigation_actions = [
    "1. IMMEDIATE DATA DEEP DIVE:",
    "   • Analyze specific work orders that were returned in May 2025",
    "   • Categorize return reasons (quality, completion, materials, etc.)",
    "   • Identify which teams/departments had the most returns",
    "",
    "2. OPERATIONAL FACTORS TO INVESTIGATE:",
    "   • Staff scheduling changes in May 2025 (new hires, resignations)",
    "   • Weekend shift coverage and supervision levels",
    "   • Friday end-of-week rush and quality control processes",
    "   • Material shortages or supply chain issues in May",
    "",
    "3. EXTERNAL FACTORS TO CHECK:",
    "   • Client deadline pressures or rush orders",
    "   • Seasonal demands or project deadlines",
    "   • Weather conditions affecting work quality",
    "   • Equipment maintenance schedules",
    "",
    "4. PROCESS ANALYSIS:",
    "   • Review quality control procedures for weekend work",
    "   • Check if Friday work is being rushed to meet deadlines",
    "   • Analyze supervisor coverage on weekends",
    "   • Review training adequacy for weekend staff",
    "",
    "5. DATA TO COLLECT:",
    "   • Detailed return reason codes",
    "   • Staff overtime hours in May 2025",
    "   • Client satisfaction scores",
    "   • Equipment downtime logs",
    "   • Supervisor presence during weekend shifts"
]

for action in investigation_actions:
    print(action)

print("\n🎯 PRIORITY FOCUS AREAS:")
print("1. Friday work quality - why 78 returns vs historical avg ~35")
print("2. Weekend supervision - Saturday had 65 returns (usually ~35)")
print("3. Month-end pressure - May often has deadline rushes")

# Create executive summary data
exec_summary = pd.DataFrame({
    'Metric': ['May 2025 Friday Returns', 'May 2025 Weekend Returns', 'Historical Friday Avg', 
               'Historical Weekend Avg', 'Friday Increase', 'Weekend Increase'],
    'Value': [may_2025_fri, may_2025_weekend_total, f"{friday_mean:.0f}", 
              f"{weekend_mean:.0f}", f"+{((may_2025_fri-friday_mean)/friday_mean*100):.0f}%",
              f"+{((may_2025_weekend_total-weekend_mean)/weekend_mean*100):.0f}%"]
})

print("\n📋 EXECUTIVE SUMMARY TABLE:")
print(exec_summary.to_string(index=False))

🚨 MAY 2025 INVESTIGATION REPORT 🚨

📊 MAY 2025 DETAILED BREAKDOWN:
day_of_week  คืนงาน (กะทันหัน)  คืนงาน (ล่วงหน้า)  total_returns
        Sat                 38                 27             65
        Fri                 52                 26             78
        Thu                 36                 19             55
        Wed                 35                 15             50
        Tue                 31                 17             48
        Mon                 28                 17             45
        Sun                 32                 16             48

🔴 MAY 2025 WEEKEND + FRIDAY TOTAL: 191 returns
   - Friday: 78 returns
   - Saturday: 65 returns
   - Sunday: 48 returns

📈 HISTORICAL COMPARISON:

All MAY months comparison:
report_month  2024-05  2025-05
day_of_week                   
Fri                63       78
Sat                37       65
Sun                34       48

🚨 KEY FINDINGS:
   - May 2024 Weekend+Friday: 134 returns
   - May 2025 Weekend+Fr


📊 STATISTICAL ANALYSIS:
Friday Analysis:
  - Historical Friday average: 54.5
  - May 2025 Friday: 78
  - Z-score: 1.48 (>2 = outlier)
  - Status: ✅ Normal

Weekend Analysis:
  - Historical weekend average: 95.7
  - May 2025 weekend: 113
  - Z-score: 1.14
  - Status: ✅ Normal

🔍 RECOMMENDED INVESTIGATION ACTIONS:
1. IMMEDIATE DATA DEEP DIVE:
   • Analyze specific work orders that were returned in May 2025
   • Categorize return reasons (quality, completion, materials, etc.)
   • Identify which teams/departments had the most returns

2. OPERATIONAL FACTORS TO INVESTIGATE:
   • Staff scheduling changes in May 2025 (new hires, resignations)
   • Weekend shift coverage and supervision levels
   • Friday end-of-week rush and quality control processes
   • Material shortages or supply chain issues in May

3. EXTERNAL FACTORS TO CHECK:
   • Client deadline pressures or rush orders
   • Seasonal demands or project deadlines
   • Weather conditions affecting work quality
   • Equipment maintena

In [6]:
import plotly.graph_objects as go

# Prepare data for May 2025
x_days = may_2025_data['day_of_week']
urgent = may_2025_data['คืนงาน (กะทันหัน)']
planned = may_2025_data['คืนงาน (ล่วงหน้า)']

fig_may2025_line = go.Figure()
fig_may2025_line.add_trace(go.Scatter(
    x=x_days, y=urgent, mode='lines+markers', name='คืนงาน (กะทันหัน)', line=dict(color='red')
))
fig_may2025_line.add_trace(go.Scatter(
    x=x_days, y=planned, mode='lines+markers', name='คืนงาน (ล่วงหน้า)', line=dict(color='orange')
))

fig_may2025_line.update_layout(
    title='May 2025: Urgent vs Planned Returns by Day',
    xaxis_title='Day of Week',
    yaxis_title='Number of Returns',
    xaxis=dict(categoryorder='array', categoryarray=day_order),
    height=400
)
fig_may2025_line.show()

In [7]:
df

,report_month,day_of_week,คืนงาน (กะทันหัน),คืนงาน (ล่วงหน้า),total_returns,year,month,month_name
2,2024-01,Sat,34,16,50,2024,01,Jan 2024
0,2024-01,Fri,36,17,53,2024,01,Jan 2024
4,2024-01,Thu,25,23,48,2024,01,Jan 2024
6,2024-01,Wed,24,19,43,2024,01,Jan 2024
5,2024-01,Tue,27,18,45,2024,01,Jan 2024
...,...,...,...,...,...,...,...,...
116,2025-05,Thu,36,19,55,2025,05,May 2025
118,2025-05,Wed,35,15,50,2025,05,May 2025
117,2025-05,Tue,31,17,48,2025,05,May 2025
113,2025-05,Mon,28,17,45,2025,05,May 2025


# **Assignment: Filter to only Urgent return and Planned return trend**

**July 8, 2025 • **

In [8]:
# Group by year and sum urgent and planned returns
yearly_summary = df.groupby('year')[['คืนงาน (กะทันหัน)', 'คืนงาน (ล่วงหน้า)']].sum().reset_index()

# Plot bar chart
import plotly.graph_objects as go

fig_yearly = go.Figure(data=[
    go.Bar(
        x=yearly_summary['year'],
        y=yearly_summary['คืนงาน (กะทันหัน)'],
        name='คืนงาน (กะทันหัน)',
        marker_color='red',
        text=yearly_summary['คืนงาน (กะทันหัน)'],  # Data labels for this bar
        textposition='auto' # Automatically position the labels (inside, outside, etc.)
    ),
    go.Bar(
        x=yearly_summary['year'],
        y=yearly_summary['คืนงาน (ล่วงหน้า)'],
        name='คืนงาน (ล่วงหน้า)',
        marker_color='orange',
        text=yearly_summary['คืนงาน (ล่วงหน้า)'],  # Data labels for this bar
        textposition='auto' # Automatically position the labels
    )
])

fig_yearly.update_layout(
    barmode='group',
    title='Yearly Return Rate (2024 vs 2025)',
    xaxis_title='Year',
    yaxis_title='Number of Returns'
)
fig_yearly.show()

In [9]:
import plotly.graph_objects as go

# Group by report_month and sum urgent, planned, and total returns
monthly_trend = df.groupby('report_month')[['คืนงาน (กะทันหัน)', 'คืนงาน (ล่วงหน้า)', 'total_returns']].sum().reset_index()

fig_monthly = go.Figure()
fig_monthly.add_trace(go.Scatter(
    x=monthly_trend['report_month'],
    y=monthly_trend['คืนงาน (กะทันหัน)'],
    mode='lines+markers',
    name='คืนงาน (กะทันหัน)',
    line=dict(color='red')
))
fig_monthly.add_trace(go.Scatter(
    x=monthly_trend['report_month'],
    y=monthly_trend['คืนงาน (ล่วงหน้า)'],
    mode='lines+markers',
    name='คืนงาน (ล่วงหน้า)',
    line=dict(color='orange')
))
fig_monthly.add_trace(go.Scatter(
    x=monthly_trend['report_month'],
    y=monthly_trend['total_returns'],
    mode='lines+markers',
    name='รวมคืนงาน',
    line=dict(color='blue')
))

fig_monthly.update_layout(
    title='Monthly Return Rate (Urgent, Planned, Total)',
    xaxis_title='Month',
    yaxis_title='Number of Returns',
    xaxis=dict(tickangle=45)
)
fig_monthly.show()

In [11]:
# Data from professional-cancel-booking-note.xlsx
data = {
    '2025-01': [0.00, 0.00, 57.02, 42.98],
    '2025-02': [0.00, 0.00, 57.93, 42.07],
    '2025-03': [5.51, 0.00, 59.15, 35.34],
    '2025-04': [16.14, 0.00, 45.50, 38.36],
    '2025-05': [10.07, 3.89, 55.84, 30.21],
    '2025-06': [7.42, 5.16, 59.03, 28.39],
}

# The row index (problem types)
index = [
    'คืนงาน (โครงการป้อนงานคุณแม่บ้าน)',
    'คืนงาน (โควิด)',
    'คืนงาน (กะทันหัน)',
    'คืนงาน (ล่วงหน้า)'
]

# Create the DataFrame
df_table = pd.DataFrame(data, index=index)

# You can optionally name the index and columns if you want them to appear
# like 'problem' and 'year' as in your image header.
df_table.index.name = 'problem'
df_table.columns.name = 'year'

# To display the table neatly (especially if it's large)
print(df_table.to_string(float_format="%.2f%%"))

year                               2025-01  2025-02  2025-03  2025-04  2025-05  2025-06
problem                                                                                
คืนงาน (โครงการป้อนงานคุณแม่บ้าน)    0.00%    0.00%    5.51%   16.14%   10.07%    7.42%
คืนงาน (โควิด)                       0.00%    0.00%    0.00%    0.00%    3.89%    5.16%
คืนงาน (กะทันหัน)                   57.02%   57.93%   59.15%   45.50%   55.84%   59.03%
คืนงาน (ล่วงหน้า)                   42.98%   42.07%   35.34%   38.36%   30.21%   28.39%


In [14]:
# Select only the required rows
df_plot = df_table.loc[['คืนงาน (กะทันหัน)', 'คืนงาน (ล่วงหน้า)']]

fig = go.Figure()

# Define the colors you want
colors = {
    'คืนงาน (กะทันหัน)': '#ffa705',  # Orange
    'คืนงาน (ล่วงหน้า)': '#02c0d7'   # Teal/Cyan
}

for idx, row in df_plot.iterrows():
    fig.add_trace(go.Bar(
        x=df_plot.columns,
        y=row.values,
        name=idx,
        text=[f"{v:.2f}%" for v in row.values],
        textposition='auto',
        marker_color=colors[idx] # Assign color based on the index (name)
    ))

fig.update_layout(
    barmode='group',
    title='Proportion of Urgent vs Planned Returns by Month',
    xaxis_title='Month',
    yaxis_title='Proportion (%)'
)
fig.show()

**Finished! ส่งรูปนี้ให้พี่ต้นน้ำได้!**

Incorrect! ต้องแก้ให้ Proportion เป็น 100% นะ

In [15]:
import plotly.graph_objects as go

# Select only Urgent and Planned rows
df_urgent_planned = df_table.loc[['คืนงาน (กะทันหัน)', 'คืนงาน (ล่วงหน้า)']]

# Calculate 100% stacked proportions for each month
proportion = df_urgent_planned.div(df_urgent_planned.sum(axis=0), axis=1) * 100

fig = go.Figure()

for idx in proportion.index:
    fig.add_trace(go.Bar(
        x=proportion.columns,
        y=proportion.loc[idx],
        name=idx,
        marker_color=colors[idx],
        text=[f"{v:.1f}%" for v in proportion.loc[idx]],
        textposition='inside'
    ))

fig.update_layout(
    barmode='stack',
    title='Proportion of Urgent vs Planned Returns by Month (100% Stacked)',
    xaxis_title='Month',
    yaxis_title='Proportion (%)',
    yaxis=dict(range=[0, 100])
)
fig.show()